In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

class GroceryShopSystem:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Star Super Market")
        self.root.geometry("1200x800")
        self.root.configure(bg='#f0f0f0')
        
        # Initialize database
        self.init_database()
        
        # Initialize ML models
        self.sales_model = None
        self.recommendation_model = None
        self.le_product = LabelEncoder()
        self.le_category = LabelEncoder()
        
        # Current cart
        self.cart = []
        self.cart_total = 0.0
        
        # Create GUI
        self.create_gui()
        
        # Load sample data and train models
        self.load_sample_data()
        self.train_models()
    
    def init_database(self):
        """Initialize SQLite database"""
        self.conn = sqlite3.connect('grocery_store.db')
        self.cursor = self.conn.cursor()
        
        # Create tables
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                category TEXT NOT NULL,
                price REAL NOT NULL,
                stock INTEGER NOT NULL,
                supplier TEXT
            )
        ''')
        
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS sales (
                id INTEGER PRIMARY KEY,
                product_id INTEGER,
                quantity INTEGER,
                total_price REAL,
                sale_date TEXT,
                customer_id INTEGER,
                FOREIGN KEY (product_id) REFERENCES products (id)
            )
        ''')
        
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS customers (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT,
                phone TEXT,
                join_date TEXT
            )
        ''')
        
        self.conn.commit()
    
    def create_gui(self):
        """Create the main GUI"""
        # Title
        title_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        title_frame.pack(fill='x')
        title_frame.pack_propagate(False)
        
        title_label = tk.Label(title_frame, text="⭐ STAR SUPER MARKET ⭐", 
                              font=('Arial', 24, 'bold'), fg='white', bg='#2c3e50')
        title_label.pack(expand=True)
        
        # Main container
        main_frame = tk.Frame(self.root, bg='#f0f0f0')
        main_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Left panel - Product management
        left_frame = tk.LabelFrame(main_frame, text="Product Management", 
                                  font=('Arial', 12, 'bold'), bg='#f0f0f0')
        left_frame.pack(side='left', fill='both', expand=True, padx=(0, 5))
        
        # Product entry form
        form_frame = tk.Frame(left_frame, bg='#f0f0f0')
        form_frame.pack(fill='x', padx=10, pady=10)
        
        tk.Label(form_frame, text="Product Name:", bg='#f0f0f0').grid(row=0, column=0, sticky='w')
        self.product_name_var = tk.StringVar()
        tk.Entry(form_frame, textvariable=self.product_name_var, width=20).grid(row=0, column=1, padx=5)
        
        tk.Label(form_frame, text="Category:", bg='#f0f0f0').grid(row=0, column=2, sticky='w')
        self.category_var = tk.StringVar()
        category_combo = ttk.Combobox(form_frame, textvariable=self.category_var, width=15)
        category_combo['values'] = ('Fruits', 'Vegetables', 'Dairy', 'Meat', 'Bakery', 'Beverages', 'Snacks')
        category_combo.grid(row=0, column=3, padx=5)
        
        tk.Label(form_frame, text="Price:", bg='#f0f0f0').grid(row=1, column=0, sticky='w')
        self.price_var = tk.StringVar()
        tk.Entry(form_frame, textvariable=self.price_var, width=20).grid(row=1, column=1, padx=5)
        
        tk.Label(form_frame, text="Stock:", bg='#f0f0f0').grid(row=1, column=2, sticky='w')
        self.stock_var = tk.StringVar()
        tk.Entry(form_frame, textvariable=self.stock_var, width=15).grid(row=1, column=3, padx=5)
        
        # Buttons
        btn_frame = tk.Frame(left_frame, bg='#f0f0f0')
        btn_frame.pack(fill='x', padx=10, pady=5)
        
        tk.Button(btn_frame, text="Add Product", command=self.add_product, 
                 bg='#27ae60', fg='white', font=('Arial', 10, 'bold')).pack(side='left', padx=2)
        tk.Button(btn_frame, text="Update Stock", command=self.update_stock, 
                 bg='#3498db', fg='white', font=('Arial', 10, 'bold')).pack(side='left', padx=2)
        tk.Button(btn_frame, text="Predict Sales", command=self.predict_sales, 
                 bg='#e74c3c', fg='white', font=('Arial', 10, 'bold')).pack(side='left', padx=2)
        
        # Product list
        self.product_tree = ttk.Treeview(left_frame, columns=('ID', 'Name', 'Category', 'Price', 'Stock'), show='headings')
        self.product_tree.heading('ID', text='ID')
        self.product_tree.heading('Name', text='Name')
        self.product_tree.heading('Category', text='Category')
        self.product_tree.heading('Price', text='Price')
        self.product_tree.heading('Stock', text='Stock')
        
        for col in ('ID', 'Name', 'Category', 'Price', 'Stock'):
            self.product_tree.column(col, width=80, anchor='center')
        
        scrollbar = ttk.Scrollbar(left_frame, orient='vertical', command=self.product_tree.yview)
        self.product_tree.configure(yscrollcommand=scrollbar.set)
        
        tree_frame = tk.Frame(left_frame)
        tree_frame.pack(fill='both', expand=True, padx=10, pady=10)
        self.product_tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        
        # Right panel - Sales and Analytics
        right_frame = tk.LabelFrame(main_frame, text="Sales & Analytics", 
                                   font=('Arial', 12, 'bold'), bg='#f0f0f0')
        right_frame.pack(side='right', fill='both', expand=True, padx=(5, 0))
        
        # Sales section
        sales_frame = tk.Frame(right_frame, bg='#f0f0f0')
        sales_frame.pack(fill='x', padx=10, pady=10)
        
        tk.Label(sales_frame, text="Quick Sale:", font=('Arial', 12, 'bold'), bg='#f0f0f0').pack(anchor='w')
        
        sale_input_frame = tk.Frame(sales_frame, bg='#f0f0f0')
        sale_input_frame.pack(fill='x', pady=5)
        
        tk.Label(sale_input_frame, text="Product ID:", bg='#f0f0f0').grid(row=0, column=0)
        self.sale_product_var = tk.StringVar()
        tk.Entry(sale_input_frame, textvariable=self.sale_product_var, width=10).grid(row=0, column=1, padx=5)
        
        tk.Label(sale_input_frame, text="Quantity:", bg='#f0f0f0').grid(row=0, column=2)
        self.sale_quantity_var = tk.StringVar()
        tk.Entry(sale_input_frame, textvariable=self.sale_quantity_var, width=10).grid(row=0, column=3, padx=5)
        
        tk.Button(sale_input_frame, text="Add to Cart", command=self.add_to_cart, 
                 bg='#f39c12', fg='white', font=('Arial', 10, 'bold')).grid(row=0, column=4, padx=5)
        
        # Cart display
        cart_frame = tk.LabelFrame(right_frame, text="Shopping Cart", bg='#f0f0f0')
        cart_frame.pack(fill='both', expand=True, padx=10, pady=5)
        
        self.cart_listbox = tk.Listbox(cart_frame, height=8)
        self.cart_listbox.pack(fill='both', expand=True, padx=5, pady=5)
        
        cart_total_frame = tk.Frame(cart_frame, bg='#f0f0f0')
        cart_total_frame.pack(fill='x', padx=5, pady=5)
        
        self.cart_total_label = tk.Label(cart_total_frame, text="Total: $0.00", 
                                        font=('Arial', 12, 'bold'), bg='#f0f0f0')
        self.cart_total_label.pack(side='left')
        
        tk.Button(cart_total_frame, text="Checkout", command=self.checkout, 
                 bg='#27ae60', fg='white', font=('Arial', 10, 'bold')).pack(side='right')
        tk.Button(cart_total_frame, text="Clear Cart", command=self.clear_cart, 
                 bg='#e74c3c', fg='white', font=('Arial', 10, 'bold')).pack(side='right', padx=(0, 5))
        
        # Analytics section
        analytics_frame = tk.LabelFrame(right_frame, text="Analytics", bg='#f0f0f0')
        analytics_frame.pack(fill='x', padx=10, pady=(0, 10))
        
        analytics_btn_frame = tk.Frame(analytics_frame, bg='#f0f0f0')
        analytics_btn_frame.pack(fill='x', padx=5, pady=5)
        
        tk.Button(analytics_btn_frame, text="Sales Report", command=self.show_sales_report, 
                 bg='#9b59b6', fg='white', font=('Arial', 10, 'bold')).pack(side='left', padx=2)
        tk.Button(analytics_btn_frame, text="Recommendations", command=self.show_recommendations, 
                 bg='#1abc9c', fg='white', font=('Arial', 10, 'bold')).pack(side='left', padx=2)
        tk.Button(analytics_btn_frame, text="Low Stock Alert", command=self.check_low_stock, 
                 bg='#e67e22', fg='white', font=('Arial', 10, 'bold')).pack(side='left', padx=2)
        
        # Refresh product list initially
        self.refresh_product_list()
    
    def load_sample_data(self):
        """Load sample data for demonstration"""
        # Check if data already exists
        self.cursor.execute("SELECT COUNT(*) FROM products")
        if self.cursor.fetchone()[0] == 0:
            # Sample products
            sample_products = [
                ('Apples', 'Fruits', 2.50, 100, 'Fresh Farms'),
                ('Bananas', 'Fruits', 1.20, 150, 'Fresh Farms'),
                ('Milk', 'Dairy', 3.80, 50, 'Dairy Co'),
                ('Bread', 'Bakery', 2.20, 30, 'Local Bakery'),
                ('Chicken Breast', 'Meat', 8.50, 25, 'Meat Masters'),
                ('Carrots', 'Vegetables', 1.80, 80, 'Fresh Farms'),
                ('Orange Juice', 'Beverages', 4.50, 40, 'Juice Plus'),
                ('Potato Chips', 'Snacks', 3.20, 60, 'Snack Attack'),
                ('Yogurt', 'Dairy', 1.50, 45, 'Dairy Co'),
                ('Tomatoes', 'Vegetables', 2.80, 70, 'Fresh Farms')
            ]
            
            for product in sample_products:
                self.cursor.execute('''
                    INSERT INTO products (name, category, price, stock, supplier) 
                    VALUES (?, ?, ?, ?, ?)
                ''', product)
            
            # Sample sales data
            import random
            for i in range(100):
                product_id = random.randint(1, 10)
                quantity = random.randint(1, 10)
                self.cursor.execute("SELECT price FROM products WHERE id = ?", (product_id,))
                price = self.cursor.fetchone()[0]
                total_price = quantity * price
                sale_date = (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d')
                customer_id = random.randint(1, 20)
                
                self.cursor.execute('''
                    INSERT INTO sales (product_id, quantity, total_price, sale_date, customer_id) 
                    VALUES (?, ?, ?, ?, ?)
                ''', (product_id, quantity, total_price, sale_date, customer_id))
            
            self.conn.commit()
    
    def train_models(self):
        """Train ML models for sales prediction and recommendations"""
        try:
            # Get sales data
            sales_df = pd.read_sql_query('''
                SELECT s.*, p.name, p.category, p.price 
                FROM sales s 
                JOIN products p ON s.product_id = p.id
            ''', self.conn)
            
            if len(sales_df) > 10:  # Ensure we have enough data
                # Prepare data for sales prediction
                sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
                sales_df['day_of_week'] = sales_df['sale_date'].dt.dayofweek
                sales_df['month'] = sales_df['sale_date'].dt.month
                
                # Encode categorical variables
                sales_df['name_encoded'] = self.le_product.fit_transform(sales_df['name'])
                sales_df['category_encoded'] = self.le_category.fit_transform(sales_df['category'])
                
                # Features for sales prediction
                X = sales_df[['product_id', 'name_encoded', 'category_encoded', 'price', 'day_of_week', 'month']]
                y = sales_df['quantity']
                
                # Train sales prediction model
                if len(X) > 5:
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                    self.sales_model = RandomForestRegressor(n_estimators=50, random_state=42)
                    self.sales_model.fit(X_train, y_train)
                    
                    predictions = self.sales_model.predict(X_test)
                    mae = mean_absolute_error(y_test, predictions)
                    print(f"Sales Prediction Model MAE: {mae:.2f}")
                
        except Exception as e:
            print(f"Error training models: {e}")
    
    def add_product(self):
        """Add a new product to inventory"""
        try:
            name = self.product_name_var.get()
            category = self.category_var.get()
            price = float(self.price_var.get())
            stock = int(self.stock_var.get())
            
            if not all([name, category, price, stock]):
                messagebox.showerror("Error", "Please fill in all fields")
                return
            
            self.cursor.execute('''
                INSERT INTO products (name, category, price, stock, supplier) 
                VALUES (?, ?, ?, ?, ?)
            ''', (name, category, price, stock, "Manual Entry"))
            
            self.conn.commit()
            self.refresh_product_list()
            
            # Clear form
            self.product_name_var.set("")
            self.category_var.set("")
            self.price_var.set("")
            self.stock_var.set("")
            
            messagebox.showinfo("Success", "Product added successfully!")
            
        except ValueError:
            messagebox.showerror("Error", "Please enter valid price and stock values")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add product: {e}")
    
    def update_stock(self):
        """Update stock for selected product"""
        selected = self.product_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select a product to update")
            return
        
        product_id = self.product_tree.item(selected[0])['values'][0]
        new_stock = simpledialog.askinteger("Update Stock", "Enter new stock quantity:")
        
        if new_stock is not None:
            self.cursor.execute("UPDATE products SET stock = ? WHERE id = ?", (new_stock, product_id))
            self.conn.commit()
            self.refresh_product_list()
            messagebox.showinfo("Success", "Stock updated successfully!")
    
    def add_to_cart(self):
        """Add product to shopping cart"""
        try:
            product_id = int(self.sale_product_var.get())
            quantity = int(self.sale_quantity_var.get())
            
            # Get product details
            self.cursor.execute("SELECT name, price, stock FROM products WHERE id = ?", (product_id,))
            result = self.cursor.fetchone()
            
            if not result:
                messagebox.showerror("Error", "Product not found")
                return
            
            name, price, stock = result
            
            if quantity > stock:
                messagebox.showerror("Error", f"Insufficient stock. Available: {stock}")
                return
            
            # Add to cart
            item_total = price * quantity
            self.cart.append({
                'id': product_id,
                'name': name,
                'price': price,
                'quantity': quantity,
                'total': item_total
            })
            
            self.cart_total += item_total
            
            # Update cart display
            self.cart_listbox.insert(tk.END, f"{name} x{quantity} - ${item_total:.2f}")
            self.cart_total_label.config(text=f"Total: ${self.cart_total:.2f}")
            
            # Clear input fields
            self.sale_product_var.set("")
            self.sale_quantity_var.set("")
            
        except ValueError:
            messagebox.showerror("Error", "Please enter valid product ID and quantity")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add to cart: {e}")
    
    def checkout(self):
        """Process checkout"""
        if not self.cart:
            messagebox.showwarning("Warning", "Cart is empty")
            return
        
        try:
            customer_id = 1  # Default customer for demo
            sale_date = datetime.now().strftime('%Y-%m-%d')
            
            for item in self.cart:
                # Record sale
                self.cursor.execute('''
                    INSERT INTO sales (product_id, quantity, total_price, sale_date, customer_id) 
                    VALUES (?, ?, ?, ?, ?)
                ''', (item['id'], item['quantity'], item['total'], sale_date, customer_id))
                
                # Update stock
                self.cursor.execute('''
                    UPDATE products SET stock = stock - ? WHERE id = ?
                ''', (item['quantity'], item['id']))
            
            self.conn.commit()
            
            messagebox.showinfo("Success", f"Checkout completed! Total: ${self.cart_total:.2f}")
            self.clear_cart()
            self.refresh_product_list()
            
            # Retrain models with new data
            self.train_models()
            
        except Exception as e:
            messagebox.showerror("Error", f"Checkout failed: {e}")
    
    def clear_cart(self):
        """Clear shopping cart"""
        self.cart = []
        self.cart_total = 0.0
        self.cart_listbox.delete(0, tk.END)
        self.cart_total_label.config(text="Total: $0.00")
    
    def refresh_product_list(self):
        """Refresh the product list display"""
        for item in self.product_tree.get_children():
            self.product_tree.delete(item)
        
        self.cursor.execute("SELECT id, name, category, price, stock FROM products")
        products = self.cursor.fetchall()
        
        for product in products:
            self.product_tree.insert('', 'end', values=product)
    
    def predict_sales(self):
        """Predict sales for next week"""
        if not self.sales_model:
            messagebox.showwarning("Warning", "Sales prediction model not trained yet")
            return
        
        try:
            predictions_window = tk.Toplevel(self.root)
            predictions_window.title("Sales Predictions")
            predictions_window.geometry("600x400")
            
            # Get all products
            self.cursor.execute("SELECT id, name, category, price FROM products")
            products = self.cursor.fetchall()
            
            predictions = []
            for product in products:
                product_id, name, category, price = product
                
                # Prepare features for prediction (next week)
                if name in self.le_product.classes_ and category in self.le_category.classes_:
                    name_encoded = self.le_product.transform([name])[0]
                    category_encoded = self.le_category.transform([category])[0]
                    
                    # Features: product_id, name_encoded, category_encoded, price, day_of_week, month
                    next_week_features = [[product_id, name_encoded, category_encoded, price, 1, datetime.now().month]]
                    predicted_quantity = self.sales_model.predict(next_week_features)[0]
                    
                    predictions.append((name, max(0, int(predicted_quantity))))
            
            # Display predictions
            predictions.sort(key=lambda x: x[1], reverse=True)
            
            text_widget = tk.Text(predictions_window, wrap=tk.WORD)
            text_widget.pack(fill='both', expand=True, padx=10, pady=10)
            
            text_widget.insert(tk.END, "SALES PREDICTIONS FOR NEXT WEEK\n")
            text_widget.insert(tk.END, "="*40 + "\n\n")
            
            for name, predicted_qty in predictions:
                text_widget.insert(tk.END, f"{name}: {predicted_qty} units\n")
                
        except Exception as e:
            messagebox.showerror("Error", f"Failed to generate predictions: {e}")
    
    def show_sales_report(self):
        """Show sales analytics"""
        report_window = tk.Toplevel(self.root)
        report_window.title("Sales Report")
        report_window.geometry("800x600")
        
        # Get sales data
        sales_df = pd.read_sql_query('''
            SELECT s.sale_date, s.quantity, s.total_price, p.name, p.category 
            FROM sales s 
            JOIN products p ON s.product_id = p.id
            ORDER BY s.sale_date DESC
        ''', self.conn)
        
        if sales_df.empty:
            tk.Label(report_window, text="No sales data available", 
                    font=('Arial', 14)).pack(expand=True)
            return
        
        # Create notebook for different views
        notebook = ttk.Notebook(report_window)
        notebook.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Daily sales tab
        daily_frame = ttk.Frame(notebook)
        notebook.add(daily_frame, text="Daily Sales")
        
        daily_sales = sales_df.groupby('sale_date')['total_price'].sum().reset_index()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(pd.to_datetime(daily_sales['sale_date']), daily_sales['total_price'], marker='o')
        ax.set_title('Daily Sales Revenue')
        ax.set_xlabel('Date')
        ax.set_ylabel('Revenue ($)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, daily_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
        
        # Category sales tab
        category_frame = ttk.Frame(notebook)
        notebook.add(category_frame, text="Category Sales")
        
        category_sales = sales_df.groupby('category')['total_price'].sum()
        
        fig2, ax2 = plt.subplots(figsize=(8, 6))
        ax2.pie(category_sales.values, labels=category_sales.index, autopct='%1.1f%%')
        ax2.set_title('Sales by Category')
        
        canvas2 = FigureCanvasTkAgg(fig2, category_frame)
        canvas2.draw()
        canvas2.get_tk_widget().pack(fill='both', expand=True)
    
    def show_recommendations(self):
        """Show product recommendations based on sales patterns"""
        rec_window = tk.Toplevel(self.root)
        rec_window.title("Product Recommendations")
        rec_window.geometry("500x400")
        
        # Simple recommendation based on sales frequency
        recommendations = pd.read_sql_query('''
            SELECT p.name, COUNT(*) as frequency, AVG(s.quantity) as avg_quantity,
                   SUM(s.total_price) as total_revenue
            FROM sales s 
            JOIN products p ON s.product_id = p.id
            GROUP BY p.name
            ORDER BY frequency DESC, total_revenue DESC
            LIMIT 10
        ''', self.conn)
        
        text_widget = tk.Text(rec_window, wrap=tk.WORD)
        text_widget.pack(fill='both', expand=True, padx=10, pady=10)
        
        text_widget.insert(tk.END, "TOP RECOMMENDED PRODUCTS\n")
        text_widget.insert(tk.END, "="*30 + "\n\n")
        text_widget.insert(tk.END, "Based on sales frequency and revenue:\n\n")
        
        for _, row in recommendations.iterrows():
            text_widget.insert(tk.END, f"• {row['name']}\n")
            text_widget.insert(tk.END, f"  Sales frequency: {row['frequency']}\n")
            text_widget.insert(tk.END, f"  Avg quantity: {row['avg_quantity']:.1f}\n")
            text_widget.insert(tk.END, f"  Total revenue: ${row['total_revenue']:.2f}\n\n")
    
    def check_low_stock(self):
        """Check for products with low stock"""
        low_stock_threshold = 20
        
        self.cursor.execute("SELECT name, stock FROM products WHERE stock < ?", (low_stock_threshold,))
        low_stock_products = self.cursor.fetchall()
        
        if not low_stock_products:
            messagebox.showinfo("Stock Status", "All products have sufficient stock!")
            return
        
        alert_window = tk.Toplevel(self.root)
        alert_window.title("Low Stock Alert")
        alert_window.geometry("400x300")
        
        tk.Label(alert_window, text="⚠️ LOW STOCK ALERT ⚠️", 
                font=('Arial', 16, 'bold'), fg='red').pack(pady=10)
        
        text_widget = tk.Text(alert_window, wrap=tk.WORD)
        text_widget.pack(fill='both', expand=True, padx=10, pady=10)
        
        text_widget.insert(tk.END, f"Products with stock below {low_stock_threshold}:\n\n")
        
        for name, stock in low_stock_products:
            text_widget.insert(tk.END, f"• {name}: {stock} units remaining\n")
    
    def run(self):
        """Start the application"""
        self.root.mainloop()
        self.conn.close()

# Create and run the application
if __name__ == "__main__":
    app = GroceryShopSystem()
    app.run()

Sales Prediction Model MAE: 2.73


In [ ]:
 # if len(X) > 5:
                   # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                   # self.sales_model = RandomForestRegressor(n_estimators=50, random_state=42)
                   # self.sales_model.fit(X_train, y_train)
                    
                   # predictions = self.sales_model.predict(X_test)
                   # mae = mean_absolute_error(y_test, predictions)
                   # print(f"Sales Prediction Model MAE: {mae:.2f}")